# Nuevos listados

En esta notebook, vamos a generar los nuevos listados de palabras...

In [17]:
%matplotlib inline

import pandas as pd

analizadas = pd.read_csv("data/listado_definitivo.csv")
analizadas.set_index(' ', inplace=True)
analizadas["candidata"] = analizadas["Palabra Candidata"]

In [18]:
palabras_ya_analizadas = analizadas[analizadas['candidata'].notna()]

print("Tenemos {} palabras analizadas".format(palabras_ya_analizadas.shape[0]))

Tenemos 5014 palabras analizadas


## Listado de Palabras

Vamos a regenerar el listado completo de palabras. Y calculemos cuánto falta trabajar si hacemos de nuevo los cálculos

In [40]:
import re

df = pd.read_csv("data/cantidades_filtradas.csv", index_col=0)
df = df[df.cantUsuariosTotal >= 5]

cant_palabras = [c for c in df.columns if re.match(r'.*Palabras$', c)]
cant_personas = [c for c in df.columns if re.match(r'.*Usuarios$', c)]

df["cant_provincias"] = (df[cant_palabras] > 0).sum(axis=1)

Calculemos el valor de la información de cada palabra.

In [41]:
from scipy.stats import entropy
import numpy as np

df["entropy_palabras"] = df[cant_palabras].apply(entropy, axis=1, raw=True)
df["entropy_personas"] = df[cant_personas].apply(entropy, axis=1, raw=True)

df["delta_palabras"] = np.log2(23) - df["entropy_palabras"]
df["delta_personas"] = np.log2(23) - df["entropy_personas"]

df["log_cantidad"] = np.log2(df.cantPalabra)
df["log_personas"] = np.log2(df.cantUsuariosTotal)

df["norm_cantidad"] = df["log_cantidad"] / df["log_cantidad"].max() 
df["norm_personas"] = df["log_personas"] / df["log_personas"].max()

df["ival_palabras"] = df.norm_cantidad * df.delta_palabras
df["ival_personas"] = df.norm_personas * df.delta_personas


df["ival"] = df["ival_palabras"] * df["ival_personas"]
df["ival_norm"] = (1+df["ival_palabras"]) * (1+df["ival_personas"])

df_personas = df.sort_values("ival_personas", ascending=False)
df_palabras = df.sort_values("ival_palabras", ascending=False)
df_pp = df.sort_values("ival", ascending=False)


Cambiamos ligeramente el cálculo de la normalización y del IV, contemplando el orden de personas

¿De las primeras 1000, cuántas no están en las ya analizadas?

In [42]:
def calcular_porcentaje_candidatas(palabras):
    subconj_analizado = palabras.index.intersection(palabras_ya_analizadas.index)
    candidatas = (analizadas.loc[subconj_analizado]['candidata'] != '0').sum()
    total = subconj_analizado.shape[0]
    print("Palabras candidatas = {}".format(candidatas))
    print("Total analizadas actualmente = {}".format(total))
    
    return candidatas / total

def analizar_palabras_restantes(n, dataframes):
    total = 0
    todas_las_palabras = set()
    print("="* 80)
    print("Primeras {} palabras".format(n))
    
    for nombre, this_df in dataframes.items(): 
        print('#'*40)
        print("Métrica: {}".format(nombre))
        palabras = this_df.iloc[:n]
        
        metrica = calcular_porcentaje_candidatas(palabras)
        restantes = {p for p in palabras.index if p not in palabras_ya_analizadas.index}
        total+=len(restantes)
        todas_las_palabras = todas_las_palabras.union(restantes)
        print("Faltan etiquetar {} palabras".format(len(restantes)))
        print("Porcentaje candidatas = {}".format(metrica))
    print("Faltan etiquetar (sin repeticiones) = {}".format(len(todas_las_palabras)))

In [43]:
dataframes = {
    "Palabras": df_palabras, 
    "Personas": df_personas, 
    "PalPer": df_pp,
}
analizar_palabras_restantes(1000, dataframes)
dataframes = {
    "Personas": df_personas, 
    "PalPer": df_pp,
}

analizar_palabras_restantes(5000, dataframes)


Primeras 1000 palabras
########################################
Métrica: Palabras
Palabras candidatas = 185
Total analizadas actualmente = 821
Faltan etiquetar 179 palabras
Porcentaje candidatas = 0.22533495736906212
########################################
Métrica: Personas
Palabras candidatas = 309
Total analizadas actualmente = 908
Faltan etiquetar 92 palabras
Porcentaje candidatas = 0.34030837004405284
########################################
Métrica: PalPer
Palabras candidatas = 250
Total analizadas actualmente = 920
Faltan etiquetar 80 palabras
Porcentaje candidatas = 0.2717391304347826
Faltan etiquetar (sin repeticiones) = 250
Primeras 5000 palabras
########################################
Métrica: Personas
Palabras candidatas = 795
Total analizadas actualmente = 1999
Faltan etiquetar 3001 palabras
Porcentaje candidatas = 0.39769884942471234
########################################
Métrica: PalPer
Palabras candidatas = 843
Total analizadas actualmente = 2420
Faltan etiquetar 258

## Sin usar el log(23)

In [44]:
# Cargo de nuevo el dataframe
df = pd.read_csv("data/cantidades_filtradas.csv", index_col=0)
df = df[df.cantUsuariosTotal >= 2]

cant_palabras = [c for c in df.columns if re.match(r'.*Palabras$', c)]
cant_personas = [c for c in df.columns if re.match(r'.*Usuarios$', c)]

df["cant_provincias"] = (df[cant_palabras] > 0).sum(axis=1)

In [45]:
from contrastes.information_value import simulated_shuffled_entropy_multinomial
np.random.seed(seed=1)

fn = lambda ws: simulated_shuffled_entropy_multinomial(int(ws), len(cant_palabras))

df["sh_entropy_palabras"] = df.cantPalabra.apply(fn)
df["sh_entropy_personas"] = df.cantUsuariosTotal.apply(fn)

df["entropy_palabras"] = df[cant_palabras].apply(entropy, axis=1, raw=True)
df["entropy_personas"] = df[cant_personas].apply(entropy, axis=1, raw=True)

df["delta_palabras"] = df["sh_entropy_palabras"] - df["entropy_palabras"]
df["delta_personas"] = df["sh_entropy_personas"] - df["entropy_personas"]

df["log_cantidad"] = np.log2(df.cantPalabra)
df["log_personas"] = np.log2(df.cantUsuariosTotal)

df["norm_cantidad"] = df["log_cantidad"] / df["log_cantidad"].max() 
df["norm_personas"] = df["log_personas"] / df["log_personas"].max()

df["ival_palabras"] = df.norm_cantidad * df.delta_palabras
df["ival_personas"] = df.norm_personas * df.delta_personas
df["ival"] = df["ival_palabras"] * df["ival_personas"]

df_personas = df.sort_values("ival_personas", ascending=False)
df_palabras = df.sort_values("ival_palabras", ascending=False)
df_pp = df.sort_values("ival", ascending=False)

In [46]:
dataframes = {
    "Palabras": df_palabras, 
    "Personas": df_personas, 
    "PalPer": df_pp
}
analizar_palabras_restantes(1000, dataframes)
dataframes = {
    "Personas": df_personas, 
    "PalPer": df_pp,
    
}

analizar_palabras_restantes(5000, dataframes)


Primeras 1000 palabras
########################################
Métrica: Palabras
Palabras candidatas = 151
Total analizadas actualmente = 782
Faltan etiquetar 218 palabras
Porcentaje candidatas = 0.19309462915601022
########################################
Métrica: Personas
Palabras candidatas = 298
Total analizadas actualmente = 1000
Faltan etiquetar 0 palabras
Porcentaje candidatas = 0.298
########################################
Métrica: PalPer
Palabras candidatas = 252
Total analizadas actualmente = 1000
Faltan etiquetar 0 palabras
Porcentaje candidatas = 0.252
Faltan etiquetar (sin repeticiones) = 218
Primeras 5000 palabras
########################################
Métrica: Personas
Palabras candidatas = 2094
Total analizadas actualmente = 4617
Faltan etiquetar 383 palabras
Porcentaje candidatas = 0.4535412605588044
########################################
Métrica: PalPer
Palabras candidatas = 1902
Total analizadas actualmente = 4496
Faltan etiquetar 504 palabras
Porcentaje candid

## Usando cálculo con normalización "vieja"


In [47]:
# Cargo de nuevo el dataframe
df = pd.read_csv("data/cantidades_filtradas.csv", index_col=0)
df = df[df.cantUsuariosTotal >= 2]

cant_palabras = [c for c in df.columns if re.match(r'.*Palabras$', c)]
cant_personas = [c for c in df.columns if re.match(r'.*Usuarios$', c)]

df["cant_provincias"] = (df[cant_palabras] > 0).sum(axis=1)

In [48]:
from contrastes.information_value import simulated_shuffled_entropy_multinomial
np.random.seed(seed=1)

fn = lambda ws: simulated_shuffled_entropy_multinomial(int(ws), len(cant_palabras))

df["sh_entropy_palabras"] = df.cantPalabra.apply(fn)
df["sh_entropy_personas"] = df.cantUsuariosTotal.apply(fn)


In [49]:
df["entropy_palabras"] = df[cant_palabras].apply(entropy, axis=1, raw=True)
df["entropy_personas"] = df[cant_personas].apply(entropy, axis=1, raw=True)

df["delta_palabras"] = df["sh_entropy_palabras"] - df["entropy_palabras"]
df["delta_personas"] = df["sh_entropy_personas"] - df["entropy_personas"]

df["log_cantidad"] = np.log2(df.cantPalabra)
df["log_personas"] = np.log2(df.cantUsuariosTotal)

df["norm_cantidad"] = (df["log_cantidad"] - df["log_cantidad"].min()) \
    / (df["log_cantidad"].max() - df["log_cantidad"].min())
df["norm_personas"] = (df["log_personas"] - df["log_personas"].min()) \
    / (df["log_personas"].max() - df["log_personas"].min())

df["ival_palabras"] = df.norm_cantidad * df.delta_palabras
df["ival_personas"] = df.norm_personas * df.delta_personas


df["ival"] = df["ival_palabras"] * df["ival_personas"]

df_personas = df.sort_values("ival_personas", ascending=False)
df_palabras = df.sort_values("ival_palabras", ascending=False)
df_pp = df.sort_values("ival", ascending=False)

In [50]:
dataframes = {
    "Palabras": df_palabras, 
    "Personas": df_personas, 
    "PalPer": df_pp
}
analizar_palabras_restantes(1000, dataframes)
dataframes = {
    "Personas": df_personas, 
    "PalPer": df_pp,
    
}

analizar_palabras_restantes(5000, dataframes)


Primeras 1000 palabras
########################################
Métrica: Palabras
Palabras candidatas = 226
Total analizadas actualmente = 772
Faltan etiquetar 228 palabras
Porcentaje candidatas = 0.2927461139896373
########################################
Métrica: Personas
Palabras candidatas = 315
Total analizadas actualmente = 1000
Faltan etiquetar 0 palabras
Porcentaje candidatas = 0.315
########################################
Métrica: PalPer
Palabras candidatas = 296
Total analizadas actualmente = 1000
Faltan etiquetar 0 palabras
Porcentaje candidatas = 0.296
Faltan etiquetar (sin repeticiones) = 228
Primeras 5000 palabras
########################################
Métrica: Personas
Palabras candidatas = 2088
Total analizadas actualmente = 4569
Faltan etiquetar 431 palabras
Porcentaje candidatas = 0.45699277741300065
########################################
Métrica: PalPer
Palabras candidatas = 1657
Total analizadas actualmente = 3875
Faltan etiquetar 1125 palabras
Porcentaje candi

## Usando el cálculo viejo CASI idéntico

No usó normalización en las dos primeras métricas

In [55]:
# Cargo de nuevo el dataframe
df = pd.read_csv("data/cantidades_filtradas.csv", index_col=0)
df = df[df.cantUsuariosTotal >= 5]

cant_palabras = [c for c in df.columns if re.match(r'.*Palabras$', c)]
cant_personas = [c for c in df.columns if re.match(r'.*Usuarios$', c)]

df["cant_provincias"] = (df[cant_palabras] > 0).sum(axis=1)

In [56]:

from contrastes.information_value import simulated_shuffled_entropy_multinomial
np.random.seed(seed=1)

fn = lambda ws: simulated_shuffled_entropy_multinomial(int(ws), len(cant_palabras))


df["sh_entropy_personas"] = df.cantUsuariosTotal.apply(fn)
df["sh_entropy_palabras"] = df.cantPalabra.apply(fn)

In [57]:
df["entropy_palabras"] = df[cant_palabras].apply(entropy, axis=1, raw=True)
df["entropy_personas"] = df[cant_personas].apply(entropy, axis=1, raw=True)

df["delta_palabras"] = df["sh_entropy_palabras"] - df["entropy_palabras"]
df["delta_personas"] = df["sh_entropy_personas"] - df["entropy_personas"]

df["log_cantidad"] = np.log2(df.cantPalabra)
df["log_personas"] = np.log2(df.cantUsuariosTotal)

df["norm_cantidad"] = (df["log_cantidad"] - df["log_cantidad"].min()) \
    / (df["log_cantidad"].max() - df["log_cantidad"].min())
df["norm_personas"] = (df["log_personas"] - df["log_personas"].min()) \
    / (df["log_personas"].max() - df["log_personas"].min())

df["ival_palabras"] = df.log_cantidad * df.delta_palabras
df["ival_personas"] = df.log_personas * df.delta_personas
df["ival"] = df.norm_cantidad * df.norm_personas * df.delta_palabras * df.delta_personas

df_personas = df.sort_values("ival_personas", ascending=False)
df_palabras = df.sort_values("ival_palabras", ascending=False)
df_pp = df.sort_values("ival", ascending=False)

In [58]:
dataframes = {
    "Palabras": df_palabras, 
    "Personas": df_personas, 
    "PalPer": df_pp
}
analizar_palabras_restantes(1000, dataframes)
dataframes = {
    "Personas": df_personas, 
    "PalPer": df_pp,
    
}

analizar_palabras_restantes(5000, dataframes)


Primeras 1000 palabras
########################################
Métrica: Palabras
Palabras candidatas = 169
Total analizadas actualmente = 857
Faltan etiquetar 143 palabras
Porcentaje candidatas = 0.1971995332555426
########################################
Métrica: Personas
Palabras candidatas = 293
Total analizadas actualmente = 1000
Faltan etiquetar 0 palabras
Porcentaje candidatas = 0.293
########################################
Métrica: PalPer
Palabras candidatas = 313
Total analizadas actualmente = 998
Faltan etiquetar 2 palabras
Porcentaje candidatas = 0.313627254509018
Faltan etiquetar (sin repeticiones) = 144
Primeras 5000 palabras
########################################
Métrica: Personas
Palabras candidatas = 2087
Total analizadas actualmente = 4621
Faltan etiquetar 379 palabras
Porcentaje candidatas = 0.4516338454879896
########################################
Métrica: PalPer
Palabras candidatas = 1647
Total analizadas actualmente = 3783
Faltan etiquetar 1217 palabras
Porcen